In [2]:
import pandas as pd
import numpy as np
import time 
from skimage import io,transform
from glob import glob
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection

# train_df = pd.read_csv('D:/Code/VSCode_Python/data/titanic_data/train.csv')
# test_df = pd.read_csv('D:/Code/VSCode_Python/data/titanic_data/test.csv')
# print(train_df.shape,test_df.shape)

# 准备数据
df = pd.read_csv("D:/Code/VSCode_Python/data/titanic_data/train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
# 特征工程
# 空数据处理（ Age、Cabin、Embarked、Fare）：

# 1. 船票价格Fare：只有一个测试数据为空，取中位数即可
df['Fare'].fillna(df['Fare'].median(), inplace = True)
# 2.登录港口Embarked：由于是非数值型，取最常见数!!!!?
df.Embarked[df.Embarked.isnull()] = df.Embarked.mode()[0]
# 3.座舱号Cabin ：由于丢失严重，补充一个特殊值‘’
df.Cabin.fillna("Unknown", inplace=True)
# 4.年龄age,先按照中位数填充
df.Age.fillna(df.Age.median(), inplace=True)



C:\Users\Raisei\AppData\Local\Temp/ipykernel_13960/839094312.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Embarked[df.Embarked.isnull()] = df.Embarked.mode()[0]


In [4]:
# 文本数据转换（Name、Ticket、Cabin）：
df["Cabin"] = df.Cabin.apply(lambda x: x[0])
tg = df.Ticket.value_counts()
df["Ticket_group"] = df.Ticket.apply(lambda x: tg[x])
df["First_Name"] = df.Name.apply(lambda x: x[: x.find(",")])
df["Title"] = df.Name.apply(lambda x: x[x.find(",") + 1 : x.find(".")])
df["Last_Name"] = df.Name.apply(lambda x: x[x.find(".") + 1 :])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   891 non-null    int64  
 1   Survived      891 non-null    int64  
 2   Pclass        891 non-null    int64  
 3   Name          891 non-null    object 
 4   Sex           891 non-null    object 
 5   Age           891 non-null    float64
 6   SibSp         891 non-null    int64  
 7   Parch         891 non-null    int64  
 8   Ticket        891 non-null    object 
 9   Fare          891 non-null    float64
 10  Cabin         891 non-null    object 
 11  Embarked      891 non-null    object 
 12  Ticket_group  891 non-null    int64  
 13  First_Name    891 non-null    object 
 14  Title         891 non-null    object 
 15  Last_Name     891 non-null    object 
dtypes: float64(2), int64(6), object(8)
memory usage: 111.5+ KB


In [5]:
# 1. Pclass：只有3种情况，直接用独热编码
# 2.sibSp、Parch ：都是家人数量，进行合并
# 3. Age、Fare ：通过直方图，观察数据特征，进行人为划分，再进行独热编码

df["Family_Size"] = df["Parch"] + df["SibSp"] + 1
titles = df["Title"]
print(titles)
# 按照身份填充缺失年龄年龄
for title in titles:
    # age_to_impute = df.groupby("Title")["Age"].median()[
    #     titles.index(title)
    # ]  # index接受Int类型
    # df.loc[(df["Age"].isnull()) & (df["Title"] == title), "Age"] = age_to_impute
    m_age = df["Age"][df["Title"] == title].median()
    df.loc[(df["Age"].isnull()) & (df["Title"] == title), "Age"] = m_age

# 增加儿童标识
df.loc[:, "Child"] = 1
df.loc[(df["Age"] >= 18), "Child"] = 0


# 名字长度与获救概率有关系吗？
df["Name_Length"] = df["Name"].apply(lambda x: len(x))


0         Mr
1        Mrs
2       Miss
3        Mrs
4         Mr
       ...  
886      Rev
887     Miss
888     Miss
889       Mr
890       Mr
Name: Title, Length: 891, dtype: object


In [6]:
# 数值数据转换（ Pclass、sibSp、Parch、 Age、Fare）：
DEFAULT_SURVIVAL_VALUE = 0.5
df["Family_Survival"] = DEFAULT_SURVIVAL_VALUE
for grp, grp_df in df[
    [
        "Survived",
        "Name",
        "First_Name",
        "Fare",
        "Ticket",
        "PassengerId",
        "SibSp",
        "Parch",
        "Age",
        "Cabin",
    ]
].groupby(["First_Name", "Fare"]):
    if len(grp_df) != 1:
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)["Survived"].max()
            smin = grp_df.drop(ind)["Survived"].min()
            passID = row["PassengerId"]
            if smax == 1.0:
                df.loc[df["PassengerId"] == passID, "Family_Survival"] = 1
            elif smin == 0.0:
                df.loc[df["PassengerId"] == passID, "Family_Survival"] = 0
            print(
                "Number of passengers with family survival information:",
                df.loc[df["Family_Survival"] != 0.75].shape[0],
            )
for _, grp_df in df.groupby("Ticket"):
    if len(grp_df) != 1:
        for ind, row in grp_df.iterrows():
            if (row["Family_Survival"] == 0) | (row["Family_Survival"] == 0.75):
                smax = grp_df.drop(ind)["Survived"].max()
                smin = grp_df.drop(ind)["Survived"].min()
                passID = row["PassengerId"]
                if smax == 1.0:
                    df.loc[df["PassengerId"] == passID, "Family_Survival"] = 1
                elif smin == 0.0:
                    df.loc[df["PassengerId"] == passID, "Family_Survival"] = 0


Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 891
Number of passengers with family survival information: 8

In [7]:
# 划分特征数据和标签数据
X = df.iloc[:, 2:]
y = df.iloc[:, 1]
X = X.drop("Name", axis=1)
X = pd.get_dummies(X)
# X=X.select_dtypes(include=np.number)
# X=X.dropna(axis=1)

In [8]:
# 训练模型
clf = DecisionTreeClassifier()
clf.fit(X, y)

DecisionTreeClassifier()

In [9]:
# 交叉验证
sc = model_selection.cross_val_score(clf, X, y, scoring="accuracy")
print(sc.mean())

0.8148327160881301


In [10]:
import sklearn.pipeline as pip
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest

pipe=pip.Pipeline([('select',SelectKBest(k=480)),('classify', RandomForestClassifier(random_state = 10, max_features = 'sqrt'))])
param_test = {
'classify__random_state':list(range(6,18,1))
}
gsearch = GridSearchCV(estimator = pipe, param_grid = param_test, scoring='accuracy', 
cv=5, verbose=10)
gsearch.fit(X, y)
print('randomforest',gsearch.best_params_, gsearch.best_score_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] classify__random_state=6 ........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
D:\Environment\Anaconda2020.11\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [  23   37   41   50   54   61   65   67   88   97  106  123  130  134
  144  159  163  164  165  191  196  198  202  209  214  216  227  231
  251  257  260  266  270  277  282  283  286  287  290  292  294  295
  300  308  309  310  313  316  323  330  335  344  353  359  366  367
  373  374  392  396  400  403  417  418  425  429  430  440  450  472
  473  474  475  482  484  485  487  503  517  519  526  528  534  535
  547  549  550  552  553  561  564  565  567  569  571  585  598  607
  608  610  613  616  619  624  629  634  635  642  649  652  664  666
  678  681  690  710  719  722  723  747  748  756  758  763  770  783
  788  798  804  807  808  813  815  817  821  831  834  836  837  838
  841  858  867  869  877  881  887  903  904  913  918  920  922  929
  956  961 

[CV] ............ classify__random_state=6, score=0.827, total=   0.6s
[CV] classify__random_state=6 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
D:\Environment\Anaconda2020.11\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [  17   28   29   32   36   42   43   46   51   75   93  102  110  114
  117  121  124  136  137  140  143  146  161  169  171  172  175  179
  181  189  194  197  210  224  225  226  239  241  243  244  254  269
  274  302  311  318  320  322  326  328  334  337  338  346  352  355
  365  377  382  386  387  388  390  394  398  424  434  453  455  457
  464  476  477  490  494  495  506  515  522  525  536  537  538  539
  542  545  558  566  582  588  601  605  606  618  627  641  647  648
  654  665  670  671  682  684  686  689  691  700  712  741  745  749
  751  759  766  768  769  776  784  791  795  800  803  812  822  824
  825  830  843  849  854  856  857  860  865  871  884  889  895  896
  902  919  927  933  936  939  942  947  953  958  959  968  972  975
  978  983  

[CV] ............ classify__random_state=6, score=0.826, total=   0.6s
[CV] classify__random_state=6 ........................................


...

[CV] ........... classify__random_state=17, score=0.820, total=   0.5s
[CV] classify__random_state=17 .......................................


D:\Environment\Anaconda2020.11\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [  20   24   26   27   30   35   38   47   63   74   77   78   94   99
  100  109  113  119  120  131  132  138  139  149  166  173  177  186
  190  203  207  211  213  217  218  220  233  234  235  240  246  248
  250  255  276  278  280  281  299  305  315  321  324  325  329  332
  339  342  348  350  351  354  361  362  363  370  371  375  376  395
  399  406  411  416  419  436  439  448  454  460  462  463  478  486
  489  493  496  497  498  514  520  523  555  575  577  592  594  602
  603  609  621  630  631  644  655  658  659  662  674  679  683  687
  705  711  713  714  718  734  735  739  740  742  743  757  760  779
  782  789  792  802  806  809  814  819  828  829  840  844  850  864
  873  876  880  882  901  907  909  914  916  917  930  931  943  950
  951  954  957  966  967  969  979  982  984  998  999 1015 1022 1028
 1035 1041 1045 1046 

[CV] ........... classify__random_state=17, score=0.865, total=   0.6s


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   34.9s finished


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('select', SelectKBest(k=480)),
                                       ('classify',
                                        RandomForestClassifier(max_features='sqrt',
                                                               random_state=10))]),
             param_grid={'classify__random_state': [6, 7, 8, 9, 10, 11, 12, 13,
                                                    14, 15, 16, 17]},
             scoring='accuracy', verbose=10)

In [11]:
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

log = LogisticRegression(penalty="l2", solver="liblinear", C=0.25)
xgb = XGBClassifier(
    learning_rate=0.01,
    n_estimators=860,
    max_depth=3,
    subsample=1,
    colsample_bytree=1,
    gamma=6,
    reg_alpha=14,
    reg_lambda=3,
)
classifier = VotingClassifier(estimators=[("XGB", xgb), ("log", log)])
classifier.fit(X, y)
accuracies = cross_val_score(classifier, X, y, cv=5)
print("5 fold cross validation accuracies {}".format(accuracies.mean()))


D:\Environment\Anaconda2020.11\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:21:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\Environment\Anaconda2020.11\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:21:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\Environment\Anaconda2020.11\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[10:21:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\Environment\Anaconda2020.11\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:21:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\Environment\Anaconda2020.11\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:22:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
5 fold cross validation accuracies 0.84509446990145
